# PEC model

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Import your melt inclusion and olivine data. Expected units are wt.%, and in the column headers only the first letter of each element should be capitalized. Iron is expected to be expressed as FeO (total), with column name 'FeO'.

In [12]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"], units="wt. %")
olivine = mp.read_olivine(olivine_file, index_col=["name"], units="wt. %")

Here's an example of the expected column header formatting:

In [16]:
print(melt.columns.values)

['SiO2' 'Al2O3' 'MgO' 'CaO' 'FeO' 'Na2O' 'K2O' 'MnO' 'TiO2' 'P2O5' 'Cr2O3'
 'CO2' 'H2O' 'F' 'S' 'Cl']


If you have data for internal inclusion pressures (in bars) you can also import those. 

In [17]:
pressure_file ="./data/pressure.csv"

pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

pressure

name
PI032-04-01    6527.087712
PI032-04-02    6828.021536
PI041-02-02    4802.872035
PI041-03-01    6958.250380
PI041-03-03    6826.596778
PI041-05-04    4013.630505
PI041-05-06    5269.481653
PI041-07-01    4969.100477
PI041-07-02    4175.590768
PI052-01-02    2388.318094
Name: 0, dtype: float64

Alternatively, if you have melt CO<sub>2</sub> (and H<sub>2</sub>O) data, you can use MagmaPandas to calculate volatile saturation pressures. First calculate temperature with some initial pressure and then iteratively calculate pressure and temperature. See the [MagmaPandas documentation](https://magmapandas.readthedocs.io/en/latest/notebooks/melt_basics.html) for more details.

In [18]:
temperature = melt.temperature(P_bar = 5e3)
pressure = melt.volatile_saturation_pressure(T_K=temperature)

while True:
    temperature = melt.temperature(P_bar=pressure)
    pressure_new = melt.volatile_saturation_pressure(T_K=temperature)
    dP = (pressure_new - pressure) / pressure_new # calculate percentage change
    pressure = pressure_new.copy()
    if (dP < 0.01).all():
        # break when all pressure have converged within 1% of previous values.
        break

pressure

Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.3s 
Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.3s 


name
PI032-04-01    6527.085981
PI032-04-02    6828.021519
PI041-02-02    4802.872035
PI041-03-01    6958.250380
PI041-03-03    6826.596776
PI041-05-04    4013.630505
PI041-05-06    5269.481653
PI041-07-01    4969.100468
PI041-07-02    4175.590767
PI052-01-02    2388.318094
dtype: float64

Here we will also use a whole-rock dataset to set up a model for predicting the initial FeO contents of our melt inclusions. For explanation of this model, please follow the [Initial FeO prediction](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) example.

In [19]:
wholerock_file = "./data/wholerock.csv"

wholerock = mp.read_melt(wholerock_file, index_col=["name"])

Let's begin by setting up the model to predict the initial FeO content of our melt inclusions according to the previous [example](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#). Here, the FeO prediction model is based on TiO<sub>2</sub>, Al<sub>2</sub>O<sub>3</sub> and CaO, which we can check by accessing their coefficients:

In [20]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

In [21]:
FeOi_predict.coefficients

intercept    11.585125
TiO2          1.463365
Al2O3        -0.230722
CaO          -0.169262
dtype: float64

To use this model during PEC correction, we need to store this model as a callable function, which we can do with the *model* attribute:

In [22]:
FeO_model = FeOi_predict.model

Let's first quickly test the model by predicting FeO contents for our uncorrected melt compositions:

In [23]:
FeO_model(melt)

name
PI032-04-01    10.196984
PI032-04-02    10.265411
PI041-02-02    10.221895
PI041-03-01    10.614069
PI041-03-03    10.716794
PI041-05-04     9.420381
PI041-05-06    11.476340
PI041-07-01    11.608516
PI041-07-02    11.451018
PI052-01-02     8.722012
dtype: float32

This all looks good, so let continue with setting up the PEC correction model.

First, preview the melt and olivine data to check everything looks ok:

In [24]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,50.028862,15.226979,5.330006,10.522019,8.59454,3.914280,0.730798,0.126266,2.669216,0.303542,NaN,0.658252,1.508379,0.082590,0.152337,0.035565
PI032-04-02,49.558861,16.041588,5.117479,10.266067,8.25471,3.782408,0.997826,0.129969,2.814807,0.356900,NaN,0.713472,1.412274,0.089010,0.175511,0.047265
PI041-02-02,49.112045,16.972038,4.864153,9.195718,10.07540,3.793512,1.079184,0.153456,2.807967,0.561756,NaN,0.464520,0.656341,0.047715,0.068108,0.021185
PI041-03-01,47.098515,17.448515,4.650640,12.154110,7.95402,3.707263,1.268282,0.101370,3.493271,0.611504,NaN,0.883720,0.324968,0.088070,0.096228,0.059958
PI041-03-03,46.478870,17.637102,4.762752,12.364033,7.84027,3.789446,1.294603,0.077221,3.617484,0.574248,NaN,0.910470,0.344988,0.090605,0.088145,0.061584


In [25]:
olivine.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO,total
name,,,,,,,,
PI032-04-01,38.204800,15.981400,44.194000,0.172665,0.234256,0.005229,0.240552,99.032906
PI032-04-02,38.638500,15.898400,43.467400,0.188024,0.219599,-0.007310,0.234622,98.639240
PI041-02-02,37.270100,20.815901,41.015099,0.104744,0.293681,0.016547,0.214939,99.731000
PI041-03-01,38.795799,15.469300,44.750999,0.180198,0.212197,0.031333,0.259919,99.699745
PI041-03-03,38.701900,15.782500,44.920799,0.164372,0.214836,0.037263,0.275049,100.096720


Make sure that each row in *melt* and *olivine* is a matching pair of melt inclusion and olivine host. Here we use the sample names stored in the indices of both dataframes:

In [26]:
print(melt.index, "\n\n", olivine.index)

Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name') 

 Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name')


We can verify that the indices are the same with the *equals* method:

In [27]:
melt.index.equals(olivine.index)

True

Check the configuration of MagmaPandas and the PEC model and make changes if you want to (see the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html)). You can change MagmaPandas settings either via the MagmaPandas object (here: mp.configuration) or the MagmaPEC object (here: mpc.configuration).

In [28]:
print(mpc.model_configuration)
print(mpc.PEC_configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................1
Melt Fe3+/Fe2+........................Sun2024
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed
#############################################


############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %)..........................0.05    
Kd convergence.............

Now we can initialise the PEC model, where we need the following data:

- **inclusions**

    inclusion major element compositions in oxide wt. % as a MagmaPandas Melt frame.

- **olivines**

    olivine major element compositions in oxide wt. % as a MagmaPandas Olivine frame.

- **P_bar**

    pressures in bar at which to run the model. You can use a fixed pressure for all inclusions, e.g. *P_bar=2e3* for 2 kbar, or indicate specific pressures per inclusion. In this example we do the latter, by passing the above imported pandas Series with internal inclusion pressures

- **FeO_target**

    Estimated initial FeO contents of melt inclusions. You can use a fixed value for all melt inclusions, e.g. FeO = 11, specific values for individual melt inclusions, stored in a pandas Series, or a predictive equation based on melt major element composition. This equation needs to be a callable function that accepts a Pandas DataFrame with melt compositions in oxide wt. % as input and return an array-like object with initial FeO contents per inclusion. In the example above we showed how to set up a function like this using MagmaPEC.

In [29]:
pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)

Now we simply run the model with the *correct* method. It runs in two stages: the first stage equilibrates Fe and Mg between melt inclusions and olivine hosts through Fe-Mg cation exchange and the second stage corrects melt inclusions back to their initial FeO contents by melting or crystallising olivine.

This method returns three objects:

- corrected melt compositions as a MagmaPandas Melt frame

- PEC extents as a pandas DataFrame
    
- checks to confirm the inclusions have equilibrated and reached FeOi

In [30]:
melts_corrected, pec, checks = pec_model.correct()


Equilibrating ... |██████████████████████████████| 100% [10/10] in 4.0s 
Equilibrating ... |██████████████████████████████| 100% [10/10] in 14.0s 


The dataframe with pec results has three columns:

- equilibration_crystallisation: 
    
    crystallisation extents during the equilibration stage

- PE_crystallisation:

    crystallisation extents during the crystallisation stage

- total_crystallisation:

    total amount of post-entrapment crystallisation

with all data in percent.

In [31]:
checks

,isothermal_equilibration,Kd_equilibration,FeO_converge
name,,,
PI032-04-01,True,True,True
PI032-04-02,True,True,True
PI041-02-02,True,True,True
PI041-03-01,True,True,True
PI041-03-03,True,True,True
PI041-05-04,True,True,True
PI041-05-06,True,True,True
PI041-07-01,True,True,True
PI041-07-02,True,True,True


In [37]:
pec

/Users/thomas/miniforge3/envs/py310/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.597656,12.6,10.001428
PI032-04-02,-2.644531,14.0,11.355652
PI041-02-02,-0.816406,1.2,0.383655
PI041-03-01,-2.214844,15.4,13.184424
PI041-03-03,-1.315430,14.0,12.684692
PI041-05-04,-2.023438,-2.0,-4.023315
PI041-05-06,-2.019531,4.2,2.179736
PI041-07-01,-2.982422,15.4,12.418433
PI041-07-02,-2.115234,13.4,11.285132


Results are also stored in the pec object and can be accessed via the *inclusions* and *olivine_corrected* attributes, while entrapment temperatures can be calculated on the fly with the *temperature* method of the *inclusions* Melt frame (see the [MagmaPandas documentation](https://magmapandas.readthedocs.io/en/latest/code_documentation.html#MagmaPandas.MagmaFrames.Melt.temperature))

In [33]:
pec_model.inclusions

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.928388,13.978963,7.777652,9.683097,10.262277,3.593329,0.670876,0.139118,2.450354,0.278653,0.000000,0.604279,1.384700,0.075818,0.139846,0.032649,100.0
PI032-04-02,48.431550,14.561043,7.703172,9.344737,10.386401,3.433502,0.905783,0.141994,2.555157,0.323978,0.000000,0.647658,1.281999,0.080799,0.159322,0.042905,100.0
PI041-02-02,49.113458,16.935988,4.945787,9.178192,10.190570,3.785418,1.076881,0.155988,2.801976,0.560557,0.000000,0.463529,0.654940,0.047613,0.067963,0.021140,100.0
PI041-03-01,45.894083,15.609977,7.358915,10.900379,10.700729,3.315877,1.134387,0.114677,3.124477,0.546946,0.000000,0.790423,0.290660,0.078772,0.086069,0.053628,100.0
PI041-03-03,45.303449,15.818615,7.213183,11.114940,10.799770,3.397905,1.160839,0.091433,3.243711,0.514915,0.000000,0.816396,0.309343,0.081243,0.079038,0.055221,100.0
PI041-05-04,47.836079,18.713147,3.717842,9.459991,9.304518,4.630323,1.617488,0.138111,2.503434,0.830776,0.000000,0.513503,0.464761,0.088350,0.122259,0.059418,100.0
PI041-05-06,46.420098,17.037212,4.740098,8.959102,11.432613,4.017493,1.422379,0.170122,3.648267,0.621916,0.000000,0.642579,0.588037,0.107838,0.128621,0.063624,100.0
PI041-07-01,45.792071,15.218118,7.342173,9.675641,11.559238,3.127597,1.271139,0.149012,3.529058,0.554321,0.000000,0.467264,1.019811,0.075026,0.163261,0.056270,100.0
PI041-07-02,45.873133,15.576167,6.961257,10.077798,11.412533,3.190203,1.360434,0.137179,3.535133,0.616178,0.000000,0.372254,0.616346,0.073589,0.140422,0.057376,100.0


In [34]:
pec_model.olivine_corrected

/Users/thomas/miniforge3/envs/py310/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.597656,12.6,10.001428
PI032-04-02,-2.644531,14.0,11.355652
PI041-02-02,-0.816406,1.2,0.383655
PI041-03-01,-2.214844,15.4,13.184424
PI041-03-03,-1.315430,14.0,12.684692
PI041-05-04,-2.023438,-2.0,-4.023315
PI041-05-06,-2.019531,4.2,2.179736
PI041-07-01,-2.982422,15.4,12.418433
PI041-07-02,-2.115234,13.4,11.285132


In [35]:
pec_model.inclusions.temperature(P_bar=pressure)

name
PI032-04-01    1488.370260
PI032-04-02    1489.638694
PI041-02-02    1419.370540
PI041-03-01    1493.690655
PI041-03-03    1490.389360
PI041-05-04    1388.259232
PI041-05-06    1423.043314
PI041-07-01    1479.905340
PI041-07-02    1472.156306
PI052-01-02    1381.196891
Name: T_K, dtype: float64